In [23]:
import numpy as np
from skopt import gp_minimize
from skopt.space import Real, Integer, Categorical
from skopt.utils import use_named_args

import matching.glema.common.utils.arg_utils as arg_utils
import matching.glema.common.utils.model_utils as model_utils
import matching.glema.evaluation.evaluate_pattern_matching as epm
from matching.glema.common.dataset import DesignPatternDataset

In [29]:
bounds = [
    Integer( 0, 10, name="param1" ),
    Categorical( [ True, False ], name="param2" ),
    Real( 0.1, 1.0, name="param3" )
]


def to_named_result( result, bounds ):
    return { p.name: r for r, p in zip( result, bounds ) }


def process( params ):
    return -params[ "param1" ]


@use_named_args( bounds )
def wrapper( **kwargs ):
    return process( kwargs )

# Run optimization
res = gp_minimize( wrapper, bounds, n_calls=20, random_state=0 )

# Print results
print( f"Best Score: {-res.fun}" )
print( f"Best Params: {to_named_result( res.x, bounds )}" )

Best Score: 10
Best Params: {'param1': np.int64(10), 'param2': np.True_, 'param3': 1.0}


In [24]:
args = arg_utils.parse_args( use_default=True )
args.dataset = "CPG_augm_large"
args.directed = False
args.anchored = True
version = model_utils.get_latest_model_version( args )
model_name = model_utils.get_model_name( args, version )
args = arg_utils.load_args( args, model_name )

args.pattern_dataset = "CPG_all"
args.normalized = True
args.test_data = True
args.batch_size = 128
args.num_workers = 1

In [26]:
dataset = DesignPatternDataset( args, max_pattern_examples=30, query_pattern=False )
source = dataset.get_sources()
patterns = dataset.get_patterns()

Loading sources from CPG_augm_large ...


100%|██████████| 1220/1220 [00:00<00:00, 3459.44it/s]


Loading sources from CPG_all ...


100%|██████████| 22072/22072 [00:05<00:00, 3759.45it/s]


In [43]:
def process_patterns( params ):
    p_patterns = patterns
    if params[ "pre_norm" ]:
        p_patterns = epm.normalize_patterns( p_patterns, max_distance=params[ "max_distance" ] )
    p_patterns = epm.normalize_patterns_by_presence( p_patterns,
                                                     num_graphs=1,
                                                     max_distance=params[ "max_distance" ],
                                                     q=params[ "q" ],
                                                     q_step=0.02,
                                                     n_min=params[ "n_min" ] )
    graph_sizes = [ ]
    for dp, graphs in p_patterns.items():
        if len( graphs ) == 0:
            graph_sizes.append( 0 )
            continue
        graph_sizes.append( graphs[ 0 ].number_of_nodes() )
    return np.var( graph_sizes )

process_patterns_bounds = [
    Categorical( [ True, False ], name="pre_norm" ),
    Integer(6, 8, name="max_distance" ),
    Integer(6, 30, name="n_min" ),
    Real( 0.5, 0.99, name="q" )
]

In [44]:
@use_named_args( process_patterns_bounds )
def process_patterns_wrapper( **kwargs ):
    print( kwargs )
    return process_patterns( kwargs )

# Run optimization
res = gp_minimize( process_patterns_wrapper, process_patterns_bounds, 
                   n_calls=50, random_state=42 )

# Print results
print( f"Best Score: {res.fun}" )
print( f"Best Params: {to_named_result( res.x, process_patterns_bounds )}" )

{'pre_norm': False, 'max_distance': np.int64(6), 'n_min': np.int64(25), 'q': 0.7924565773937786}
Normalizing patterns to connected ...


100%|██████████| 7/7 [00:00<00:00, 28.76it/s]


Filter connected patterns by presence ...


100%|██████████| 7/7 [00:00<00:00, 3320.91it/s]


{'pre_norm': True, 'max_distance': np.int64(6), 'n_min': np.int64(17), 'q': 0.6635172194581207}
Normalizing patterns ...


...

Normalizing patterns to connected ...


100%|██████████| 7/7 [00:00<00:00, 180.81it/s]


Filter connected patterns by presence ...


100%|██████████| 7/7 [00:00<00:00, 12132.28it/s]


Best Score: 32.244897959183675
Best Params: {'pre_norm': np.False_, 'max_distance': np.int64(6), 'n_min': np.int64(6), 'q': 0.8906684694905629}
